In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
spark = SparkSession.builder.appName("Smart City Traffic Monitoring System").getOrCreate()
spark

In [0]:
df_infra = spark.read.csv("dbfs:/FileStore/shared_uploads/azuser3559_mml.local@techademy.com/traffic_logs.csv", header=True, inferSchema=True)
df_infra.show()

+-----+---------+----------+---------+-------------------+-------------------+-----------+--------+--------+
|LogID|VehicleID|EntryPoint|ExitPoint|          EntryTime|           ExitTime|VehicleType|SpeedKMH|TollPaid|
+-----+---------+----------+---------+-------------------+-------------------+-----------+--------+--------+
| L001|     V001|     GateA|    GateC|2024-05-01 08:01:00|2024-05-01 08:20:00|        Car|      60|      50|
| L002|     V002|     GateB|    GateC|2024-05-01 08:10:00|2024-05-01 08:45:00|      Truck|      45|     100|
| L003|     V003|     GateA|    GateD|2024-05-01 09:00:00|2024-05-01 09:18:00|       Bike|      55|      30|
| L004|     V004|     GateC|    GateD|2024-05-01 09:15:00|2024-05-01 09:35:00|        Car|      80|      50|
| L005|     V005|     GateB|    GateA|2024-05-01 10:05:00|2024-05-01 10:40:00|        Bus|      40|      70|
+-----+---------+----------+---------+-------------------+-------------------+-----------+--------+--------+



In [0]:
schema = StructType([
    StructField("LogID", StringType(), True),
    StructField("VehicleID", StringType(), True),
    StructField("EntryPoint", StringType(), True),
    StructField("ExitPoint", StringType(), True),
    StructField("EntryTime", TimestampType(), True),
    StructField("ExitTime", TimestampType(), True),
    StructField("VehicleType", StringType(), True),
    StructField("SpeedKMH", IntegerType(), True),
    StructField("TollPaid", IntegerType(), True)
])

df_mun = spark.read.csv("dbfs:/FileStore/shared_uploads/azuser3559_mml.local@techademy.com/traffic_logs.csv", header=True, inferSchema=True)
df_mun.show()


+-----+---------+----------+---------+-------------------+-------------------+-----------+--------+--------+
|LogID|VehicleID|EntryPoint|ExitPoint|          EntryTime|           ExitTime|VehicleType|SpeedKMH|TollPaid|
+-----+---------+----------+---------+-------------------+-------------------+-----------+--------+--------+
| L001|     V001|     GateA|    GateC|2024-05-01 08:01:00|2024-05-01 08:20:00|        Car|      60|      50|
| L002|     V002|     GateB|    GateC|2024-05-01 08:10:00|2024-05-01 08:45:00|      Truck|      45|     100|
| L003|     V003|     GateA|    GateD|2024-05-01 09:00:00|2024-05-01 09:18:00|       Bike|      55|      30|
| L004|     V004|     GateC|    GateD|2024-05-01 09:15:00|2024-05-01 09:35:00|        Car|      80|      50|
| L005|     V005|     GateB|    GateA|2024-05-01 10:05:00|2024-05-01 10:40:00|        Bus|      40|      70|
+-----+---------+----------+---------+-------------------+-------------------+-----------+--------+--------+



In [0]:
df_infra = df_infra.withColumn("EntryTime", to_timestamp("EntryTime"))    .withColumn("ExitTime", to_timestamp("ExitTime"))

df_infra.show()
    

+-----+---------+----------+---------+-------------------+-------------------+-----------+--------+--------+
|LogID|VehicleID|EntryPoint|ExitPoint|          EntryTime|           ExitTime|VehicleType|SpeedKMH|TollPaid|
+-----+---------+----------+---------+-------------------+-------------------+-----------+--------+--------+
| L001|     V001|     GateA|    GateC|2024-05-01 08:01:00|2024-05-01 08:20:00|        Car|      60|      50|
| L002|     V002|     GateB|    GateC|2024-05-01 08:10:00|2024-05-01 08:45:00|      Truck|      45|     100|
| L003|     V003|     GateA|    GateD|2024-05-01 09:00:00|2024-05-01 09:18:00|       Bike|      55|      30|
| L004|     V004|     GateC|    GateD|2024-05-01 09:15:00|2024-05-01 09:35:00|        Car|      80|      50|
| L005|     V005|     GateB|    GateA|2024-05-01 10:05:00|2024-05-01 10:40:00|        Bus|      40|      70|
+-----+---------+----------+---------+-------------------+-------------------+-----------+--------+--------+



In [0]:
df_trip = df_infra.withColumn("TripDurationMinutes", (col("ExitTime").cast("long") - col("EntryTime").cast("long")) / 60)
df_trip.show()

+-----+---------+----------+---------+-------------------+-------------------+-----------+--------+--------+-------------------+
|LogID|VehicleID|EntryPoint|ExitPoint|          EntryTime|           ExitTime|VehicleType|SpeedKMH|TollPaid|TripDurationMinutes|
+-----+---------+----------+---------+-------------------+-------------------+-----------+--------+--------+-------------------+
| L001|     V001|     GateA|    GateC|2024-05-01 08:01:00|2024-05-01 08:20:00|        Car|      60|      50|               19.0|
| L002|     V002|     GateB|    GateC|2024-05-01 08:10:00|2024-05-01 08:45:00|      Truck|      45|     100|               35.0|
| L003|     V003|     GateA|    GateD|2024-05-01 09:00:00|2024-05-01 09:18:00|       Bike|      55|      30|               18.0|
| L004|     V004|     GateC|    GateD|2024-05-01 09:15:00|2024-05-01 09:35:00|        Car|      80|      50|               20.0|
| L005|     V005|     GateB|    GateA|2024-05-01 10:05:00|2024-05-01 10:40:00|        Bus|      4

In [0]:
df_over = df_trip.withColumn("IsOverspeed", col("SpeedKMH") > 60)
df_over.show()

+-----+---------+----------+---------+-------------------+-------------------+-----------+--------+--------+-------------------+-----------+
|LogID|VehicleID|EntryPoint|ExitPoint|          EntryTime|           ExitTime|VehicleType|SpeedKMH|TollPaid|TripDurationMinutes|IsOverspeed|
+-----+---------+----------+---------+-------------------+-------------------+-----------+--------+--------+-------------------+-----------+
| L001|     V001|     GateA|    GateC|2024-05-01 08:01:00|2024-05-01 08:20:00|        Car|      60|      50|               19.0|      false|
| L002|     V002|     GateB|    GateC|2024-05-01 08:10:00|2024-05-01 08:45:00|      Truck|      45|     100|               35.0|      false|
| L003|     V003|     GateA|    GateD|2024-05-01 09:00:00|2024-05-01 09:18:00|       Bike|      55|      30|               18.0|      false|
| L004|     V004|     GateC|    GateD|2024-05-01 09:15:00|2024-05-01 09:35:00|        Car|      80|      50|               20.0|       true|
| L005|     V

In [0]:
df_avg = df_over.groupBy("VehicleType").agg(avg("SpeedKMH").alias("AvgSpeed"))
df_avg.show()

+-----------+--------+
|VehicleType|AvgSpeed|
+-----------+--------+
|       Bike|    55.0|
|        Car|    70.0|
|      Truck|    45.0|
|        Bus|    40.0|
+-----------+--------+



In [0]:
df_toll = df_over.groupBy("VehicleType").agg(sum("TollPaid").alias("TotalTollPaid"))
df_toll.show()

+-----------+-------------+
|VehicleType|TotalTollPaid|
+-----------+-------------+
|       Bike|           30|
|        Car|          100|
|      Truck|          100|
|        Bus|           70|
+-----------+-------------+



In [0]:
df_used = df_over.groupBy("VehicleType").agg(max("ExitPoint").alias("MostUsedExitPoint"))
df_used.show()

+-----------+-----------------+
|VehicleType|MostUsedExitPoint|
+-----------+-----------------+
|       Bike|            GateD|
|        Car|            GateD|
|      Truck|            GateC|
|        Bus|            GateA|
+-----------+-----------------+



In [0]:
from pyspark.sql.window import Window

win_rank = Window.partitionBy("VehicleType").orderBy(col("SpeedKMH").desc())
df_rank = df_over.withColumn("SpeedRank", rank().over(win_rank))
df_rank.show()

+-----+---------+----------+---------+-------------------+-------------------+-----------+--------+--------+-------------------+-----------+---------+
|LogID|VehicleID|EntryPoint|ExitPoint|          EntryTime|           ExitTime|VehicleType|SpeedKMH|TollPaid|TripDurationMinutes|IsOverspeed|SpeedRank|
+-----+---------+----------+---------+-------------------+-------------------+-----------+--------+--------+-------------------+-----------+---------+
| L003|     V003|     GateA|    GateD|2024-05-01 09:00:00|2024-05-01 09:18:00|       Bike|      55|      30|               18.0|      false|        1|
| L005|     V005|     GateB|    GateA|2024-05-01 10:05:00|2024-05-01 10:40:00|        Bus|      40|      70|               35.0|      false|        1|
| L004|     V004|     GateC|    GateD|2024-05-01 09:15:00|2024-05-01 09:35:00|        Car|      80|      50|               20.0|       true|        1|
| L001|     V001|     GateA|    GateC|2024-05-01 08:01:00|2024-05-01 08:20:00|        Car|    

In [0]:
win_lag = Window.partitionBy("VehicleID").orderBy(col("ExitTime"))
df_lag = df_rank.withColumn("LastExitTime", lag("ExitTime", 1).over(win_lag))
df_lag.show()


+-----+---------+----------+---------+-------------------+-------------------+-----------+--------+--------+-------------------+-----------+---------+------------+
|LogID|VehicleID|EntryPoint|ExitPoint|          EntryTime|           ExitTime|VehicleType|SpeedKMH|TollPaid|TripDurationMinutes|IsOverspeed|SpeedRank|LastExitTime|
+-----+---------+----------+---------+-------------------+-------------------+-----------+--------+--------+-------------------+-----------+---------+------------+
| L001|     V001|     GateA|    GateC|2024-05-01 08:01:00|2024-05-01 08:20:00|        Car|      60|      50|               19.0|      false|        2|        NULL|
| L002|     V002|     GateB|    GateC|2024-05-01 08:10:00|2024-05-01 08:45:00|      Truck|      45|     100|               35.0|      false|        1|        NULL|
| L003|     V003|     GateA|    GateD|2024-05-01 09:00:00|2024-05-01 09:18:00|       Bike|      55|      30|               18.0|      false|        1|        NULL|
| L004|     V004

In [0]:
df_sess = df_lag.withColumn("IdleTimeMinutes", (col("EntryTime").cast("long") - col("LastExitTime").cast("long")) / 60)
df_sess.show()

+-----+---------+----------+---------+-------------------+-------------------+-----------+--------+--------+-------------------+-----------+---------+------------+---------------+
|LogID|VehicleID|EntryPoint|ExitPoint|          EntryTime|           ExitTime|VehicleType|SpeedKMH|TollPaid|TripDurationMinutes|IsOverspeed|SpeedRank|LastExitTime|IdleTimeMinutes|
+-----+---------+----------+---------+-------------------+-------------------+-----------+--------+--------+-------------------+-----------+---------+------------+---------------+
| L001|     V001|     GateA|    GateC|2024-05-01 08:01:00|2024-05-01 08:20:00|        Car|      60|      50|               19.0|      false|        2|        NULL|           NULL|
| L002|     V002|     GateB|    GateC|2024-05-01 08:10:00|2024-05-01 08:45:00|      Truck|      45|     100|               35.0|      false|        1|        NULL|           NULL|
| L003|     V003|     GateA|    GateD|2024-05-01 09:00:00|2024-05-01 09:18:00|       Bike|      55| 

In [0]:
df_sess.filter((col("SpeedKMH") > 70) & (col("TripDurationMinutes") < 10)).show()

+-----+---------+----------+---------+---------+--------+-----------+--------+--------+-------------------+-----------+---------+------------+---------------+
|LogID|VehicleID|EntryPoint|ExitPoint|EntryTime|ExitTime|VehicleType|SpeedKMH|TollPaid|TripDurationMinutes|IsOverspeed|SpeedRank|LastExitTime|IdleTimeMinutes|
+-----+---------+----------+---------+---------+--------+-----------+--------+--------+-------------------+-----------+---------+------------+---------------+
+-----+---------+----------+---------+---------+--------+-----------+--------+--------+-------------------+-----------+---------+------------+---------------+



In [0]:
df_sess.filter((col("TripDurationMinutes") > 30) & (col("TollPaid") < 50)).show()

+-----+---------+----------+---------+---------+--------+-----------+--------+--------+-------------------+-----------+---------+------------+---------------+
|LogID|VehicleID|EntryPoint|ExitPoint|EntryTime|ExitTime|VehicleType|SpeedKMH|TollPaid|TripDurationMinutes|IsOverspeed|SpeedRank|LastExitTime|IdleTimeMinutes|
+-----+---------+----------+---------+---------+--------+-----------+--------+--------+-------------------+-----------+---------+------------+---------------+
+-----+---------+----------+---------+---------+--------+-----------+--------+--------+-------------------+-----------+---------+------------+---------------+



In [0]:
df_sess.filter(col("ExitPoint") < col("EntryPoint")).show()


+-----+---------+----------+---------+-------------------+-------------------+-----------+--------+--------+-------------------+-----------+---------+------------+---------------+
|LogID|VehicleID|EntryPoint|ExitPoint|          EntryTime|           ExitTime|VehicleType|SpeedKMH|TollPaid|TripDurationMinutes|IsOverspeed|SpeedRank|LastExitTime|IdleTimeMinutes|
+-----+---------+----------+---------+-------------------+-------------------+-----------+--------+--------+-------------------+-----------+---------+------------+---------------+
| L005|     V005|     GateB|    GateA|2024-05-01 10:05:00|2024-05-01 10:40:00|        Bus|      40|      70|               35.0|      false|        1|        NULL|           NULL|
+-----+---------+----------+---------+-------------------+-------------------+-----------+--------+--------+-------------------+-----------+---------+------------+---------------+



In [0]:
df_reg = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/shared_uploads/azuser3559_mml.local@techademy.com/vehicle_registry.csv")
df_reg.show()


+---------+---------+-----------+--------------+
|VehicleID|OwnerName|      Model|RegisteredCity|
+---------+---------+-----------+--------------+
|     V001|     Anil|Hyundai i20|         Delhi|
|     V002|   Rakesh| Tata Truck|       Chennai|
|     V003|     Sana| Yamaha R15|        Mumbai|
|     V004|     Neha| Honda City|     Bangalore|
|     V005|     Zoya|  Volvo Bus|          Pune|
+---------+---------+-----------+--------------+



In [0]:
df_join = df_sess.join(df_reg, on = "VehicleID")
df_join.show()

+---------+-----+----------+---------+-------------------+-------------------+-----------+--------+--------+-------------------+-----------+---------+------------+---------------+---------+-----------+--------------+
|VehicleID|LogID|EntryPoint|ExitPoint|          EntryTime|           ExitTime|VehicleType|SpeedKMH|TollPaid|TripDurationMinutes|IsOverspeed|SpeedRank|LastExitTime|IdleTimeMinutes|OwnerName|      Model|RegisteredCity|
+---------+-----+----------+---------+-------------------+-------------------+-----------+--------+--------+-------------------+-----------+---------+------------+---------------+---------+-----------+--------------+
|     V001| L001|     GateA|    GateC|2024-05-01 08:01:00|2024-05-01 08:20:00|        Car|      60|      50|               19.0|      false|        2|        NULL|           NULL|     Anil|Hyundai i20|         Delhi|
|     V002| L002|     GateB|    GateC|2024-05-01 08:10:00|2024-05-01 08:45:00|      Truck|      45|     100|               35.0|    

In [0]:
df_join.groupBy("RegisteredCity").count().show() 

+--------------+-----+
|RegisteredCity|count|
+--------------+-----+
|     Bangalore|    1|
|       Chennai|    1|
|        Mumbai|    1|
|          Pune|    1|
|         Delhi|    1|
+--------------+-----+



In [0]:
df_sess.write.format("delta").mode("overwrite").save("/delta/traffic_logs")
spark.sql("CREATE TABLE IF NOT EXISTS traffic_logs USING DELTA LOCATION '/delta/traffic_logs'")
spark.sql("SELECT * FROM traffic_logs").show()

+-----+---------+----------+---------+-------------------+-------------------+-----------+--------+--------+-------------------+-----------+---------+------------+---------------+
|LogID|VehicleID|EntryPoint|ExitPoint|          EntryTime|           ExitTime|VehicleType|SpeedKMH|TollPaid|TripDurationMinutes|IsOverspeed|SpeedRank|LastExitTime|IdleTimeMinutes|
+-----+---------+----------+---------+-------------------+-------------------+-----------+--------+--------+-------------------+-----------+---------+------------+---------------+
| L001|     V001|     GateA|    GateC|2024-05-01 08:01:00|2024-05-01 08:20:00|        Car|      60|      50|               19.0|      false|        2|        NULL|           NULL|
| L002|     V002|     GateB|    GateC|2024-05-01 08:10:00|2024-05-01 08:45:00|      Truck|      45|     100|               35.0|      false|        1|        NULL|           NULL|
| L003|     V003|     GateA|    GateD|2024-05-01 09:00:00|2024-05-01 09:18:00|       Bike|      55| 

In [0]:
from delta.tables import DeltaTable

delta_tbl = DeltaTable.forPath(spark, "/delta/traffic_logs")
spark.read.format("delta").load("/delta/traffic_logs").show()


+-----+---------+----------+---------+-------------------+-------------------+-----------+--------+--------+-------------------+-----------+---------+------------+---------------+
|LogID|VehicleID|EntryPoint|ExitPoint|          EntryTime|           ExitTime|VehicleType|SpeedKMH|TollPaid|TripDurationMinutes|IsOverspeed|SpeedRank|LastExitTime|IdleTimeMinutes|
+-----+---------+----------+---------+-------------------+-------------------+-----------+--------+--------+-------------------+-----------+---------+------------+---------------+
| L001|     V001|     GateA|    GateC|2024-05-01 08:01:00|2024-05-01 08:20:00|        Car|      60|      50|               19.0|      false|        2|        NULL|           NULL|
| L002|     V002|     GateB|    GateC|2024-05-01 08:10:00|2024-05-01 08:45:00|      Truck|      45|     100|               35.0|      false|        1|        NULL|           NULL|
| L003|     V003|     GateA|    GateD|2024-05-01 09:00:00|2024-05-01 09:18:00|       Bike|      55| 

In [0]:
delta_tbl.update(condition="VehicleType = 'Bike'",set={"TollPaid": "TollPaid + 10"})

In [0]:
delta_tbl.delete("TripDurationMinutes > 60")


In [0]:
spark.sql("DESCRIBE HISTORY traffic_logs")
spark.read.format("delta").option("versionAsOf", 0).load("/delta/traffic_logs").show()

+-----+---------+----------+---------+-------------------+-------------------+-----------+--------+--------+-------------------+-----------+---------+------------+---------------+
|LogID|VehicleID|EntryPoint|ExitPoint|          EntryTime|           ExitTime|VehicleType|SpeedKMH|TollPaid|TripDurationMinutes|IsOverspeed|SpeedRank|LastExitTime|IdleTimeMinutes|
+-----+---------+----------+---------+-------------------+-------------------+-----------+--------+--------+-------------------+-----------+---------+------------+---------------+
| L001|     V001|     GateA|    GateC|2024-05-01 08:01:00|2024-05-01 08:20:00|        Car|      60|      50|               19.0|      false|        2|        NULL|           NULL|
| L002|     V002|     GateB|    GateC|2024-05-01 08:10:00|2024-05-01 08:45:00|      Truck|      45|     100|               35.0|      false|        1|        NULL|           NULL|
| L003|     V003|     GateA|    GateD|2024-05-01 09:00:00|2024-05-01 09:18:00|       Bike|      55| 

In [0]:
df_type = df_join.withColumn("TripType", when(col("TripDurationMinutes") < 15, "Short").when(col("TripDurationMinutes") <= 30, "Medium").otherwise("Long"))
df_join.show()

+---------+-----+----------+---------+-------------------+-------------------+-----------+--------+--------+-------------------+-----------+---------+------------+---------------+---------+-----------+--------------+
|VehicleID|LogID|EntryPoint|ExitPoint|          EntryTime|           ExitTime|VehicleType|SpeedKMH|TollPaid|TripDurationMinutes|IsOverspeed|SpeedRank|LastExitTime|IdleTimeMinutes|OwnerName|      Model|RegisteredCity|
+---------+-----+----------+---------+-------------------+-------------------+-----------+--------+--------+-------------------+-----------+---------+------------+---------------+---------+-----------+--------------+
|     V001| L001|     GateA|    GateC|2024-05-01 08:01:00|2024-05-01 08:20:00|        Car|      60|      50|               19.0|      false|        2|        NULL|           NULL|     Anil|Hyundai i20|         Delhi|
|     V002| L002|     GateB|    GateC|2024-05-01 08:10:00|2024-05-01 08:45:00|      Truck|      45|     100|               35.0|    

In [0]:
df_t = df_type.withColumn("TripDate", to_date(col("EntryTime")))
df_tc =  df_t.groupBy("VehicleID", "TripDate").agg(count("*").alias("TripCount"))
df_tc3 = df_tc.filter(col("TripCount") > 3)
df_tc3.show()


+---------+--------+---------+
|VehicleID|TripDate|TripCount|
+---------+--------+---------+
+---------+--------+---------+



In [0]:
df_par = df_sess.write.partitionBy("VehicleType").mode("overwrite").parquet("/output/traffic_parquet")

In [0]:
df_sess.write.mode("overwrite").option("header", True).csv("/output/traffic_csv")
df_sess.createOrReplaceTempView("traffic_summary")
spark.sql("""
    SELECT VehicleType, ExitPoint, SUM(TollPaid) AS TotalToll
    FROM traffic_summary
    GROUP BY VehicleType, ExitPoint
""").show()

+-----------+---------+---------+
|VehicleType|ExitPoint|TotalToll|
+-----------+---------+---------+
|        Car|    GateD|       50|
|      Truck|    GateC|      100|
|       Bike|    GateD|       30|
|        Bus|    GateA|       70|
|        Car|    GateC|       50|
+-----------+---------+---------+

